## Distribute Workload using ZeroMQ

In this section, we will be using ZeroMQ to distribute workload of generating temporal traces
among distributed workers. Doing so, gives us the ability to get a much higher throughput of
simulations.

In [1]:
import sys
import time
import random
from threading import Thread
import struct

import zmq
from tqdm import tqdm
import numpy as np

port = "5556"


context = zmq.Context()
socket = context.socket(zmq.DEALER)
socket.setsockopt(zmq.IDENTITY, b'master')
socket_addr = "tcp://127.0.0.1:%s" % port
socket.bind(socket_addr)

poller = zmq.Poller()
poller.register(socket, zmq.POLLIN)

In [8]:
total_sims = 1000

def sender(num):
    # send the tasks
    for _ in range(num):
        request = b"HI"
        socket.send(request)

st = Thread(target=sender, args=(total_sims,))
st.start()

In [9]:
pbar = tqdm(total=int(total_sims))

received_sims = 0
results = []
# receive the results
while received_sims < total_sims:
    socks = dict(poller.poll(timeout=30000))
    if socks == {}:
        print("Timeout!")
        break

    if socket in socks and socks[socket] == zmq.POLLIN:
        # print("Message from socket: %s" % struct.unpack("d", socket.recv()))
        results.append(struct.unpack("d", socket.recv()))
        received_sims += 1
        pbar.update(1)

pbar.close()

100%|██████████| 1000/1000 [00:09<00:00, 109.16it/s]


In [10]:
p_cold = np.mean(results)
print(f"The probability of cold startis: {p_cold:.8f}")

The probability of cold startis: 0.00035226
